In [4]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [5]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [6]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [9]:
symbol='AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2198725447696,
 'week52high': 139.86,
 'week52low': 56.9,
 'week52change': 0.6510275702684347,
 'sharesOutstanding': 17034878172,
 'float': 0,
 'avg10Volume': 96442076,
 'avg30Volume': 109249220,
 'day200MovingAvg': 120.66,
 'day50MovingAvg': 131.76,
 'employees': 0,
 'ttmEPS': 3.38,
 'ttmDividendRate': 0.8104458841442246,
 'dividendYield': 0.00649405724233821,
 'nextDividendDate': '0',
 'exDividendDate': '2020-11-01',
 'nextEarningsDate': '0',
 'peRatio': 37.56417258070703,
 'beta': 1.162538339359476,
 'maxChangePercent': 50.18007121842559,
 'year5ChangePercent': 4.93208336157246,
 'year2ChangePercent': 2.401554293852514,
 'year1ChangePercent': 0.6266154652640237,
 'ytdChangePercent': -0.03797605115781026,
 'month6ChangePercent': 0.3338364671476075,
 'month3ChangePercent': 0.10732733682006006,
 'month1ChangePercent': 0.009669723296242492,
 'day30ChangePercent': 0.009321844619478683,
 'day5ChangePercent': -0.007542047086702559}

In [10]:
data['year1ChangePercent']

0.6266154652640237

In [11]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [12]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series([
                                                            symbol,
                                                            data[symbol]['quote']['latestPrice'],
                                                            data[symbol]['stats']['year1ChangePercent'],
                                                            'N/A'
                                                            ],
                                                           index = my_columns),
                                                            ignore_index = True)
        
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,128.96,0.444609,N/A
1,AAL,16.32,-0.451325,N/A
2,AAP,173.15,0.109661,N/A
3,AAPL,131.27,0.641892,N/A
4,ABBV,113.80,0.358167,N/A
...,...,...,...,...
500,YUM,110.60,0.0396201,N/A
501,ZBH,164.11,0.0813927,N/A
502,ZBRA,425.73,0.611332,N/A
503,ZION,50.42,0.00921249,N/A


In [13]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,41.69,2.39499,N/A
1,FCX,33.17,1.51205,N/A
2,ALB,193.33,1.38584,N/A
3,LB,47.56,1.32417,N/A
4,PYPL,251.95,1.16905,N/A
5,NVDA,534.92,1.10235,N/A
6,ALGN,583.39,1.0342,N/A
7,WST,314.78,0.933707,N/A
8,LRCX,596.00,0.924024,N/A
9,PWR,78.27,0.918453,N/A


In [14]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:100000000
100000000


In [17]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0,len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/final_dataframe['Price'][i])
final_dataframe

c:\users\pranit\anaconda3\envs\algotrade\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,CARR,41.69,2.39499,47032
1,FCX,33.17,1.51205,59113
2,ALB,193.33,1.38584,10142
3,LB,47.56,1.32417,41227
4,PYPL,251.95,1.16905,7782
5,NVDA,534.92,1.10235,3665
6,ALGN,583.39,1.0342,3361
7,WST,314.78,0.933707,6229
8,LRCX,596.00,0.924024,3289
9,PWR,78.27,0.918453,25051


In [24]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [26]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0


for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0       0.877228
1      0.0158416
2       0.542574
3       0.942574
4       0.829703
         ...    
500     0.429703
501     0.518812
502     0.938614
503     0.370297
504     0.627723
Name: One-Year Return Percentile, Length: 505, dtype: object
0      0.722772
1      0.689109
2      0.392079
3      0.669307
4      0.380198
         ...   
500    0.481188
501    0.465347
502    0.841584
503    0.887129
504    0.336634
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      0.633663
1      0.718812
2      0.368317
3      0.421782
4      0.792079
         ...   
500    0.441584
501    0.479208
502    0.833663
503    0.954455
504    0.239604
Name: Three-Month Return Percentile, Length: 505, dtype: object
0       0.69505
1      0.156436
2      0.556436
3      0.390099
4      0.746535
         ...   
500    0.211881
501    0.639604
502    0.754455
503    0.946535
504     0.30297
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.50,N/A,0.42618,0.877228,0.36742,0.722772,0.212203,0.633663,0.0719603,0.69505,N/A
1,AAL,16.21,N/A,-0.451092,0.0158416,0.346694,0.689109,0.277308,0.718812,-0.0342407,0.156436,N/A
2,AAP,169.41,N/A,0.108876,0.542574,0.155017,0.392079,0.0866363,0.368317,0.0386021,0.556436,N/A
3,AAPL,133.40,N/A,0.635979,0.942574,0.338624,0.669307,0.108428,0.421782,0.00961611,0.390099,N/A
4,ABBV,113.68,N/A,0.354301,0.829703,0.14573,0.380198,0.356928,0.792079,0.0869666,0.746535,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,110.75,N/A,0.0400225,0.429703,0.203375,0.481188,0.114144,0.441584,-0.0220909,0.211881,N/A
501,ZBH,162.65,N/A,0.0828067,0.518812,0.193773,0.465347,0.128388,0.479208,0.055258,0.639604,N/A
502,ZBRA,406.97,N/A,0.624989,0.938614,0.491681,0.841584,0.405864,0.833663,0.0883914,0.754455,N/A
503,ZION,51.71,N/A,0.00905038,0.370297,0.564746,0.887129,0.667163,0.954455,0.207827,0.946535,N/A


In [27]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [28]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

In [29]:
portfolio_input()

Enter the value of your portfolio:10000000


In [30]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

c:\users\pranit\anaconda3\envs\algotrade\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,132.50,1479,0.42618,0.877228,0.36742,0.722772,0.212203,0.633663,0.0719603,0.69505,0.732178
1,AAL,16.21,12096,-0.451092,0.0158416,0.346694,0.689109,0.277308,0.718812,-0.0342407,0.156436,0.39505
2,AAP,169.41,1157,0.108876,0.542574,0.155017,0.392079,0.0866363,0.368317,0.0386021,0.556436,0.464851
3,AAPL,133.40,1469,0.635979,0.942574,0.338624,0.669307,0.108428,0.421782,0.00961611,0.390099,0.605941
4,ABBV,113.68,1724,0.354301,0.829703,0.14573,0.380198,0.356928,0.792079,0.0869666,0.746535,0.687129
5,ABC,106.80,1835,0.15832,0.617822,-0.000914973,0.124752,0.0657737,0.326733,0.0500641,0.617822,0.421782
6,ABMD,342.00,573,0.872881,0.978218,0.189753,0.453465,0.204383,0.613861,0.15217,0.89505,0.735149
7,ABT,112.71,1739,0.290561,0.778218,0.144091,0.372277,0.0482885,0.29505,0.0381651,0.554455,0.5
8,ACN,263.18,745,0.250315,0.736634,0.164142,0.407921,0.134968,0.491089,-0.0365309,0.140594,0.444059
9,ADBE,476.80,411,0.3166,0.792079,0.0567091,0.205941,-0.0797601,0.0534653,-0.0963925,0.019802,0.267822


In [31]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [32]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [33]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [34]:
writer.save()